In [1]:
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA

import torch

from importlib import reload

import clustering
import model
import train
import utils.data

reload(clustering)
reload(utils.data)
reload(model)
reload(train)

from clustering import get_cluster_centers, cosine_dist
from model import FFN
from train import train
from utils.data import load_data_from_tsv

In [2]:
ffn = FFN(128, 16, 4, 32)
ffn.load_state_dict(torch.load("checkpoints/checkpoint_128_16_4_32_epoch=399_batch-size=8.cpkt"))

<All keys matched successfully>

In [3]:
train_df = load_data_from_tsv("dataset/train.tsv")
cluster_centers = get_cluster_centers(train_df, ffn)

In [4]:
test_df = load_data_from_tsv("dataset/test.tsv")
test_embeddings = ffn(torch.stack(list(test_df["hist"])))

good = 0
for name, embedding in zip(test_df["obj_name"], test_embeddings):
    items = list(cluster_centers.items())
    keys = [key for key, _ in items]
    dists = [float(cosine_dist(embedding, value)) for _, value in items]
    cluster_name = keys[np.argmin(dists)]
    print(f"{name} vs {cluster_name}")
    good += (name == cluster_name)
    
print(f"accuracy={good / len(test_df)}")
        

NameError: name 'np' is not defined

In [ ]:
test_embeddings_np = test_embeddings.detach().numpy()

pca = PCA(n_components=2)
pca.fit(test_embeddings_np)

test_projections = test_embeddings_np @ pca.components_.T

In [ ]:
n_classes = 5
aclust = AgglomerativeClustering(n_clusters=n_classes, affinity='cosine', linkage='complete')
aclust.fit(test_embeddings_np)

classes = [[] for i in range(n_classes)]

for i in range(len(test_df)):
    classes[aclust.labels_[i]].append(test_projections[i])

for class_ in classes:
    class_ = np.array(class_)
    plt.scatter(class_[:, 0], class_[:, 1])

plt.legend([f"class {j}" for j in range(n_classes)])

In [ ]:
queries_classes = [[] for i in range(n_classes)]
for i, query in enumerate(test_df["obj_name"]):
    queries_classes[aclust.labels_[i]].append(query)

for i, queries_class in enumerate(queries_classes):
    print(f"Class #{i}")
    for query in queries_class:
        print(query)
    print()